# SPARQL Intro

> Welcome!
> 
> Please download this notebook and load it into your [PAWS instance](https://hub-paws.wmcloud.org/)

This notebook provides an overview of the [SPARQL query language](https://www.w3.org/TR/sparql11-query/), with Wikidata queries being ran with the Python package [SPARQLWrapper](https://github.com/RDFLib/sparqlwrapper).

## Introduction

### Data structure

[Wikidata](https://www.wikidata.org/) and other [Wikibase](https://wikiba.se/) instances are not relational databases, but rather [RDF (Resource Description Framework)](https://en.wikipedia.org/wiki/Resource_Description_Framework) graph databases known as [triplestores](https://en.wikipedia.org/wiki/Triplestore). RDF is a directed graph composed of triple statements that include:

1. A subject (the entity being related)
2. A predicate (the relation between the subject and object)
3. An object (the entity being related to)

Note that objects can be a literal value (int, string, date, etc) **OR** another entity within the graph. In [Wikidata](https://www.wikidata.org/) subjects and non-literal objects are generally stored as [QIDs](https://www.wikidata.org/wiki/Q43649390) and predicates are stored as PIDs (see the [Further resources](#further-resources) section for the documentation for [Wikidata](https://www.wikidata.org/) identifiers).

A few examples of triples are the following:

- **Germany** (subject - [Q183](https://www.wikidata.org/wiki/Q183)) has the **capital** (predicate - [P36](https://www.wikidata.org/wiki/Property:P36)) **Berlin** (object - [Q64](https://www.wikidata.org/wiki/Q64)).
- **Berlin** (subject - [Q64](https://www.wikidata.org/wiki/Q64)) has a **population** of (predicate - [P1082](https://www.wikidata.org/wiki/Property:P1082)) **3.7 million** (object - an integer).
- **The European Union** (subject - [Q458](https://www.wikidata.org/wiki/Q458)) has the **member** (predicate - [P527](https://www.wikidata.org/wiki/Property:P527)) **Germany** (object - [Q183](https://www.wikidata.org/wiki/Q183)).
- **Germany** (subject - [Q183](https://www.wikidata.org/wiki/Q183)) is a **member of** (predicate - [P463](https://www.wikidata.org/wiki/Property:P463)) **the European Union** (object - [Q458](https://www.wikidata.org/wiki/Q458)).

One of the main benefits of RDF triplestores is that there are no limits based on the current structure of the data. If a new relationship is needed, then a predicate for it can be made and the associated objects can then be linked to their subjects.

When comparing to conventional data structures, it's important to mark the distinction that [Wikidata](https://www.wikidata.org/) data is not stored in tables. There are [regular dumps of Wikidata](https://www.wikidata.org/wiki/Wikidata:Database_download#RDF_dumps) that also come in relational database forms (with `subject`, `predicate` and `object` columns) as well as JSON and other types, but the data on [Wikidata](https://www.wikidata.org/) is stored using RDF relationships.

### SPARQL

Because the structure of [Wikidata](https://www.wikidata.org/) data is different from traditional relational databases, we also need a different way to query it. [SPARQL](https://en.wikipedia.org/wiki/SPARQL) - the [recursive acronym](https://en.wikipedia.org/wiki/Recursive_acronym) being "SPARQL Protocol and RDF Query Language" - is a standard of querying RDF formatted data.

> **Note**: We would strongly suggest that VS Code developers download the [Wikidata QID Labels VS Code extension](https://marketplace.visualstudio.com/items?itemName=blokhinnv.wikidataqidlabels) that provides an in editor tooltip for Wikidata ID labels.

## Queries

Below we find the most common [Wikidata](https://www.wikidata.org/) example of [Q42 - Douglas Adams](https://www.wikidata.org/wiki/Q42), who was specifically given this in homage to his book [The Hitchhiker's Guide to the Galaxy](https://en.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy) in which the "Ultimate Question of Life, the Universe, and Everything" is found to be the number 42 :)

In [1]:
from IPython.display import Image

Image(
    url="https://upload.wikimedia.org/wikipedia/commons/a/ae/Datamodel_in_Wikidata.svg"
)

You can either go to the [Wikidata Query Service](https://query.wikidata.org/) to try out the following queries or pass them to [SPARQLWrapper](https://github.com/RDFLib/sparqlwrapper) in the cells to get information about Douglas Adams. You can also click the section header to go directly to the query service with the query populated.

### Imports

In [2]:
# pip install sparqlwrapper

In [3]:
# pip install jupyter-black

In [4]:
%load_ext jupyter_black

In [5]:
from SPARQLWrapper import JSON, POST, SPARQLWrapper

In [6]:
# Set SPARQLWrapper query conditions.
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setMethod(POST)

### [Books that Douglas Adams (Q42) is the author (P50) of](https://w.wiki/8n6T)

It's important to note that for triples where the object is a [Wikidata](https://www.wikidata.org/) entity the response to queries is its unique ID, not the string label. In order to get labels for our results we need to add in the labeling service to our queries that will then give us the ability to create any `colNameLabel` column for a column of IDs `colName`. We add this service via the following line that sets English as the default returned value at the end:

```
SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
}
```

Note that `?colNameDescription` functions in a similar way where the description of the ID can be returned.

Query:

In [7]:
q42_books_query = """
SELECT
    ?book
    ?bookLabel
    ?bookDescription

WHERE {
    # subject  # author  # Douglas Adams
    ?book      wdt:P50   wd:Q42.
    
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
    }
}
"""

In [8]:
sparql.setQuery(q42_books_query)
q42_books_results = sparql.query().convert()

In [9]:
q42_books_bindings = q42_books_results["results"]["bindings"]

In [10]:
q42_books_formatted = []
for r in q42_books_bindings:
    r_dict = {k: r[k]["value"] for k in r.keys()}

    q42_books_formatted.append(r_dict)

In [11]:
q42_books_formatted[:3]

[{'book': 'http://www.wikidata.org/entity/Q721',
  'bookLabel': 'Life, the Universe and Everything',
  'bookDescription': '1982 novel by Douglas Adams'},
 {'book': 'http://www.wikidata.org/entity/Q25169',
  'bookLabel': "The Hitchhiker's Guide to the Galaxy pentalogy",
  'bookDescription': '1979-1992 series of five books by Douglas Adams'},
 {'book': 'http://www.wikidata.org/entity/Q187655',
  'bookLabel': 'Mostly Harmless',
  'bookDescription': '1992 novel by Douglas Adams'}]

### [Douglas Adams' (Q42) date of birth (P569)](https://w.wiki/8n6V)

> **Note**: We don't need to call the label service in this query as the object is a date and not a Wikidata entity.

Query:

In [12]:
q42_dob_query = """
SELECT
    ?dateOfBirth

WHERE {
    # Douglas Adams  # date of birth  # object
    wd:Q42           wdt:P569         ?dateOfBirth.
}
"""

In [13]:
sparql.setQuery(q42_dob_query)
q42_dob_results = sparql.query().convert()

In [14]:
q42_dob_bindings = q42_dob_results["results"]["bindings"]

In [15]:
q42_dob_bindings[0]["dateOfBirth"]["value"].split("T")[0]

'1952-03-11'

### [Douglas Adams' (Q42) place of birth (P19)](https://w.wiki/8n6X)

Query:

In [16]:
q42_pob_query = """
SELECT
    ?placeOfBirth
    ?placeOfBirthLabel

WHERE {
    # Douglas Adams  # place of birth  # object
    wd:Q42           wdt:P19           ?placeOfBirth.
    
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
    }
}
"""

In [17]:
sparql.setQuery(q42_pob_query)
q42_pob_results = sparql.query().convert()

In [18]:
q42_pob_bindings = q42_pob_results["results"]["bindings"]

In [19]:
q42_pob_bindings[0]["placeOfBirthLabel"]["value"]

'Cambridge'

### [All people (Q5) with the same place of birth (P19) as Douglas Adams (Q42)](https://w.wiki/8n6Z)

Query:

In [20]:
q42_same_pob_query = """
SELECT DISTINCT
    ?person
    ?personLabel
    ?personDescription

WHERE {
    # Douglas Adams  # place of birth  # object
    wd:Q42           wdt:P19           ?placeOfBirth.

    # subject  # instance of  # human
    ?person    wdt:P31        wd:Q5;
               # place of birth/*  # object
               wdt:P19/wdt:P131*   ?placeOfBirth;

    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en"
    }
}
"""

In [21]:
sparql.setQuery(q42_same_pob_query)
q42_same_pob_results = sparql.query().convert()

In [22]:
q42_same_pob_bindings = q42_same_pob_results["results"]["bindings"]

In [23]:
q42_same_pob_formatted = []
for r in q42_same_pob_bindings:
    r_dict = {k: r[k]["value"] for k in r.keys()}

    q42_same_pob_formatted.append(r_dict)

In [24]:
q42_same_pob_formatted[:3]

[{'person': 'http://www.wikidata.org/entity/Q314659',
  'personLabel': 'Rupert Friend',
  'personDescription': 'British actor'},
 {'person': 'http://www.wikidata.org/entity/Q345373',
  'personLabel': 'Bob Klose',
  'personDescription': 'British photographer'},
 {'person': 'http://www.wikidata.org/entity/Q691216',
  'personLabel': 'Julian Huppert',
  'personDescription': 'British Liberal Democrat politician (born 1978)'}]

### [All countries that are members of (P463) the European Union (Q458)](https://w.wiki/8n6b)

Here's another query to try out on the [Wikidata Query Service](https://query.wikidata.org). Can you change it to get different results? The following are great ways to find the [Wikidata](https://www.wikidata.org/) IDs you're looking for to rewrite the query below:

- Search for the main item on [Wikidata](https://www.wikidata.org/) (in this case [the European Union](https://www.wikidata.org/wiki/Q458))
  - Check statements on the left and navigate to their PIDs
- Use a search engine to search for `Wikidata NAME_OF_ITEM`, with the first result normally being the correct one
- Use the [Wikidata Query Builder](https://www.wikidata.org/wiki/Wikidata:Query_Builder) to construct your query from normal language

Query:

In [25]:
eu_countries_query = """
SELECT
    ?country
    ?countryLabel

WHERE {
    # subject  # member of  # European Union
    ?country   wdt:P463     wd:Q458.
    
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en".
    }
}
"""

In [26]:
sparql.setQuery(eu_countries_query)
eu_countries_results = sparql.query().convert()

In [27]:
eu_countries_bindings = eu_countries_results["results"]["bindings"]

In [28]:
eu_countries_formatted = []
for r in eu_countries_bindings:
    r_dict = {k: r[k]["value"] for k in r.keys()}

    eu_countries_formatted.append(r_dict)

In [29]:
eu_countries_formatted[:3]

[{'country': 'http://www.wikidata.org/entity/Q27',
  'countryLabel': 'Republic of Ireland'},
 {'country': 'http://www.wikidata.org/entity/Q28', 'countryLabel': 'Hungary'},
 {'country': 'http://www.wikidata.org/entity/Q29', 'countryLabel': 'Spain'}]

### [All present day soverign states (Q3624078) and their capitals (P36)](https://w.wiki/8n6g)

Try also to rewrite this final query to get something different!

In [30]:
modern_counries_capitals_query = """
SELECT DISTINCT 
    ?country 
    ?countryLabel 
    ?capital 
    ?capitalLabel

WHERE {
    # subject  # instance of  # sovereign state
    ?country   wdt:P31        wd:Q3624078 .
    
    # Not a former country
    FILTER NOT EXISTS {
        # subject  # instance of  # former country
        ?country   wdt:P31        wd:Q3024240
    }

    # Needed to exclude Ancient Egypt.
    FILTER NOT EXISTS {
        # subject  # instance of  # ancient civilization
        ?country   wdt:P31        wd:Q28171280
    }

    OPTIONAL {
        # subject  # capital  # object
        ?country   wdt:P36    ?capital
    } .

    SERVICE wikibase:label { 
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en"
    }
}

ORDER BY 
    ?countryLabel
"""

In [31]:
sparql.setQuery(modern_counries_capitals_query)
modern_counries_capitals_results = sparql.query().convert()

In [32]:
modern_counries_capitals_bindings = modern_counries_capitals_results["results"][
    "bindings"
]

In [33]:
modern_counries_capitals_formatted = []
for r in modern_counries_capitals_bindings:
    r_dict = {k: r[k]["value"] for k in r.keys()}

    modern_counries_capitals_formatted.append(r_dict)

In [34]:
modern_counries_capitals_formatted[:3]

[{'country': 'http://www.wikidata.org/entity/Q889',
  'countryLabel': 'Afghanistan',
  'capital': 'http://www.wikidata.org/entity/Q5838',
  'capitalLabel': 'Kabul'},
 {'country': 'http://www.wikidata.org/entity/Q222',
  'countryLabel': 'Albania',
  'capital': 'http://www.wikidata.org/entity/Q19689',
  'capitalLabel': 'Tirana'},
 {'country': 'http://www.wikidata.org/entity/Q262',
  'countryLabel': 'Algeria',
  'capital': 'http://www.wikidata.org/entity/Q3561',
  'capitalLabel': 'Algiers'}]

## Further resources

The following are other resources that we would suggests to broaden your understanding of [Wikidata](https://www.wikidata.org/). Some resources from above are repeated to assure that the this section is a comprehensive list.

### Wikidata documentation

- [Wikidata on Wikipedia](https://en.wikipedia.org/wiki/Wikidata)
- [Wikidata Identifiers](https://www.wikidata.org/wiki/Wikidata:Identifiers)

### Querying Wikidata

- [Wikidata SPARQL tutorial](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial)
- [Wikidata tutorial by Wikimedia Israel](https://wdqs-tutorial.toolforge.org/)
- [Wikidata example SPARQL queries](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)
- [Wikidata Query Builder](https://www.wikidata.org/wiki/Wikidata:Query_Builder)
- [Wikidata Query Builder Documentation](https://query.wikidata.org/querybuilder)
- [PAWS examples and recipes](https://wikitech.wikimedia.org/wiki/PAWS/PAWS_examples_and_recipes)